In [280]:
import re
import numpy as np
import pandas as pd
from datasets import load_dataset
pd.set_option('display.max_colwidth', 1000)

In [281]:
# discount = pd.DataFrame(columns=['prev_intent3','prev_intent2','prev_intent1','discount%'])
# discount

In [282]:
# intents = ['intro','inquiry','init-price','counter-price','disagree']

In [283]:
# SingleIntents = ['init-price','intro','inquiry']

# for i in intents:
#     for j in intents:
#         for k in intents:
#             arr = [i,j,k]
#             if arr.count('init-price') < 2 and arr.count('intro') < 2:
#                 discount = discount.append({'prev_intent3': i, 'prev_intent2': j, 'prev_intent1': k}, ignore_index=True)

# discount

In [284]:
biddingData = load_dataset('craigslist_bargains', split= 'train')
biddingData = pd.DataFrame(biddingData)
biddingData.rename(columns={'utterance':'bargain_convo','dialogue_acts':'intent'}, inplace=True)

Using custom data configuration default
Reusing dataset craigslist_bargains (/Users/kabirbatra/.cache/huggingface/datasets/craigslist_bargains/default/1.1.0/9e4e64d13ea36e435a8f56f9305d3c771115909b7fe83269bf7cd8dd40302338)


In [285]:
def priceExtraction(value):
    #searching for price-value or price-range in user input text string
    low,high = map(float, re.findall('[\d\.]+', value))
    return low,high

In [286]:
biddingData['price_range'] = biddingData['agent_info'].apply(lambda x: priceExtraction(str(x)))
biddingData

,agent_info,agent_turn,intent,bargain_convo,items,price_range
0,"{'Bottomline': ['None', 'None'], 'Role': ['buyer', 'seller'], 'Target': [7.0, 10.0]}","[0, 1, 0, 1, 0, 1, 0, 0, 1]","{'intent': ['init-price', 'unknown', 'insist', 'counter-price', 'counter-price', 'counter-price', 'agree', 'offer', 'accept'], 'price': [5.0, -1.0, 5.0, 8.0, 6.0, 7.0, 7.0, 7.0, -1.0]}","[Hi, not sure if the charger would work for my car. Can you sell it to me for $5?, It will work, i have never seen a car without a cigarette lighter port.\, Still, can I buy it for $5? I'm on a tight budge, I think the lowest I would want to go is 8. , How about $6 and I pick it up myself? It'll save you shipping to me., 7, and we have a deal., Eh, fine. $7., , ]","{'Category': ['phone', 'phone'], 'Images': ['phone/6149527852_0.jpg', 'phone/6149527852_0.jpg'], 'Price': [10.0, 10.0], 'Description': ['Charge two devices simultaneously on the go. This vehicle charger with an additional USB port delivers enough power to charge two devices at once. The push-button activated LED connector light means no more fumbling in the dark trying to connect your device. Auto Detect IC Technology automatically detects the device type and its specific charging needs for improved compatibility. And the built-in indicator light illuminates red to let you know the charger is receiving power and the power socket is working properly.', 'Charge two devices simultaneously on the go. This vehicle charger with an additional USB port delivers enough power to charge two devices at once. The push-button activated LED connector light means no more fumbling in the dark trying to connect your device. Auto Detect IC Technology automatically detects the device type and its spec...","(7.0, 10.0)"
1,"{'Bottomline': ['None', 'None'], 'Role': ['buyer', 'seller'], 'Target': [120.0, 200.0]}","[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1]","{'intent': ['intro', 'unknown', 'inquiry', 'disagree', 'init-price', 'vague-price', 'counter-price', 'counter-price', 'counter-price', 'agree', 'agree', 'offer', 'accept'], 'price': [-1.0, -1.0, -1.0, -1.0, 100.0, -1.0, 120.0, 150.0, 145.0, -1.0, -1.0, 145.0, -1.0]}","[Hi,I am interested in your board!, Hi, do you have any questions?, Do you know what wood it is made out of?, I do not know specifically but the brand is a gravity skateboard. , The wheels seem nice on it, but they could be a beter quality. Would you accept 100 for the board?, No, that offer is too low. The board is pretty much brand new as it's been ridden only 4 or 5 times. The bone bearings are brand new., If I picked it up at your location would you accpet 120?, Make it 150 and we got a deal. , I could do 145, if you can gurantee the painting on the front of the board is not scratched up. , Deal, Great!, , ]","{'Category': ['bike', 'bike'], 'Images': ['bike/6151236842_0.jpg', 'bike/6151236842_0.jpg'], 'Price': [200.0, 200.0], 'Description': ['Hey there cl I have a long board for sale pretty much brand knew rode all together 4 or 5 times . The brand on the deck is a gravity skateboard. The brand of trucks are sidewinder 2sGuwuing.mini monster hawgs 70mm 78a Wheels with Swiss bones bearings brand new. Please no low ballers thank you and have a great day .', 'Hey there cl I have a long board for sale pretty much brand knew rode all together 4 or 5 times . The brand on the deck is a gravity skateboard. The brand of trucks are sidewinder 2sGuwuing.mini monster hawgs 70mm 78a Wheels with Swiss bones bearings brand new. Please no low ballers thank you and have a great day .'], 'Title': ['Long board custom bilt from the board up', 'Long board custom bilt from the board up']}","(120.0, 200.0)"
2,"{'Bottomline': ['None', 'None'], 'Role': ['buyer', 'seller'], 'Target': [1920.0, 3200.0]}","[0, 1, 0, 1, 0, 1, 1, 0, 1, 0]","{'intent': ['', '', '', '', '', '', '', '', '', ''], 'price': [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]}","[i would love to buy , Sure! What's your price?, im on a budget so i could d

In [287]:
def mapIntent(value):
    intents = re.findall('[a-z\-]{2,}', value)
    # print(intents[:-2])
    if intents.count('') == len(intents):
        return np.NaN
    else:
        return intents[:-2]

def mapBids(value):
    bids = list(map(float, re.findall('[\d\.\-]{2,}', value)))
    # print(bids)
    if bids.count(-1.0) == len(bids):
        return np.NaN
    else:
        return bids

biddingData['intents'] = biddingData['intent'].apply(lambda x: mapIntent(str(x)))
biddingData['bids'] = biddingData['intent'].apply(lambda x: mapBids(str(x)))

In [288]:
biddingData.dropna(subset = ['bids'], inplace=True)
biddingData.drop(columns=['agent_info','intent','items'])

,agent_turn,bargain_convo,price_range,intents,bids
0,"[0, 1, 0, 1, 0, 1, 0, 0, 1]","[Hi, not sure if the charger would work for my car. Can you sell it to me for $5?, It will work, i have never seen a car without a cigarette lighter port.\, Still, can I buy it for $5? I'm on a tight budge, I think the lowest I would want to go is 8. , How about $6 and I pick it up myself? It'll save you shipping to me., 7, and we have a deal., Eh, fine. $7., , ]","(7.0, 10.0)","[intent, init-price, unknown, insist, counter-price, counter-price, counter-price, agree, offer]","[5.0, -1.0, 5.0, 8.0, 6.0, 7.0, 7.0, 7.0, -1.0]"
1,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1]","[Hi,I am interested in your board!, Hi, do you have any questions?, Do you know what wood it is made out of?, I do not know specifically but the brand is a gravity skateboard. , The wheels seem nice on it, but they could be a beter quality. Would you accept 100 for the board?, No, that offer is too low. The board is pretty much brand new as it's been ridden only 4 or 5 times. The bone bearings are brand new., If I picked it up at your location would you accpet 120?, Make it 150 and we got a deal. , I could do 145, if you can gurantee the painting on the front of the board is not scratched up. , Deal, Great!, , ]","(120.0, 200.0)","[intent, intro, unknown, inquiry, disagree, init-price, vague-price, counter-price, counter-price, counter-price, agree, agree, offer]","[-1.0, -1.0, -1.0, -1.0, 100.0, -1.0, 120.0, 150.0, 145.0, -1.0, -1.0, 145.0, -1.0]"
3,"[0, 1, 0, 1, 0, 1]","[hello, Hi, How old is the couch?, It is practically brand new, As you know, the value goes down as soon as the couch goes out the door. I see the list price is 585. I can give you 400 for it and can pick it up today., Well as I said it has nearly no wear and was in a non-smoking home, I can go as low as 480 on it]","(444.0, 585.0)","[intent, intro, unknown, inquiry, inform, init-price]","[-1.0, -1.0, -1.0, -1.0, 400.0, 480.0]"
6,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1]","[man thats a nice trak, yes its really nice, how about you give it to me for 8000, That is way too little , i realy dont need it but i can pay you 8200 if you let it go, I can go down to 14k but I can't knock off 6k, oh man i cant afford that, i need to buy food for my pigs and farm and send my child to private school, can you do 13k ?, I will take 13k , ok i can pick it up this weekend, , sounds good, ]","(8700.0, 14500.0)","[intent, intro, unknown, init-price, unknown, counter-price, counter-price, counter-price, agree, unknown, offer, unknown]","[-1.0, -1.0, 8000.0, -1.0, 8200.0, 14000.0, 13000.0, 13000.0, -1.0, 13000.0, -1.0, -1.0]"
14,"[0, 1, 0, 1, 0, 1, 1, 0]","[Hi, I'd love to have this sofa, but I just can't pay that much for a used one. Would you take $50?, how about 120 , I just can't go that high. I can't even see what it looks like under that blanket and you stated that you bought it used so IDK were it's been, Well 90 is lowest I will go, could you do 75 cash? That's all I've got right now, I appreciate the best offer, Its a deal, , ]","(95.0, 125.0)","[intent, init-price, counter-price, disagree, counter-price, counter-price, unknown, offer]","[50.0, 120.0, -1.0, 90.0, 75.0, -1.0, 75.0, -1.0]"
...,...,...,...,...,...
5242,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1]","[Hello I was interested in your 2011 fusion hybrid. , Hello. Great! It is a nice little car , It looks fairly decent, does everything run well? It is a bit pricey for me at the moment. Would you be able to work with me on this price a bit?, Yes, it is excellent condition with a few minor scratches. What do you have in mind?, I am searching in the ballpark of $5600. I dont know how feasible this would be for you. Maybe we could work out a deal. , Prehaps. I can do 6000. , I could work with 6000.00. When may I meet with you to finalize this?, Tomorrow afternoon , , Great Ill see you then. , Thank you, ]","(5596.0, 7995.0)","[intent, intro, unknown, inquiry, inquiry, init-price, c

In [289]:
def mapIntentWithBids(agent_turn, intents, bids):
    resIntent = []
    resBid = []
    
    for i in range(len(agent_turn)):
        if intents[i] not in ['unknown','offer','vague-price']: 
            if agent_turn[i] == 0:
                if intents[i] == 'intent':
                    resIntent.append('intro')
                else:
                    resIntent.append(intents[i])
            else:
                resBid.append(bids[i])
    
    return resIntent, resBid

biddingData['Intent_bids'] = biddingData.apply(lambda x: mapIntentWithBids(x['agent_turn'],x['intents'],x['bids']), axis=1)

In [290]:
biddingData = biddingData[['Intent_bids','price_range']]
biddingData

,Intent_bids,price_range
0,"([intro, counter-price, counter-price, agree], [-1.0, 8.0, 7.0])","(7.0, 10.0)"
1,"([intro, disagree, counter-price, agree, agree], [-1.0, -1.0, -1.0, 150.0, -1.0])","(120.0, 200.0)"
3,"([intro, inform], [-1.0, -1.0, 480.0])","(444.0, 585.0)"
6,"([intro, counter-price, agree], [-1.0, -1.0, 14000.0, 13000.0])","(8700.0, 14500.0)"
14,"([intro, counter-price, counter-price], [120.0, 90.0, -1.0])","(95.0, 125.0)"
...,...,...
5242,"([intro, inquiry, counter-price, inform], [-1.0, -1.0, 6000.0, -1.0])","(5596.0, 7995.0)"
5243,"([intro, disagree, counter-price], [-1.0, -1.0, 250.0])","(175.0, 350.0)"
5244,"([intro, disagree, inform, counter-price, counter-price], [-1.0, -1.0, 150.0, 130.0])","(105.0, 150.0)"
5245,"([intro, disagree, agree], [-1.0, -1.0, 35.0, -1.0])","(36.0, 40.0)"


In [291]:
def calDiscount(bids,price_range):
    bids = [x for x in bids if x != -1.0]
    if len(bids)!=0 and min(bids) >= price_range[0]:
        # print(min(bids),price_range[0])
        return int((abs((min(bids)-price_range[1]))/price_range[1]) * 100)
    else:
        return np.NaN

biddingData['discount%'] = biddingData.apply(lambda x: calDiscount(x['Intent_bids'][1],x['price_range']),axis=1)

<ipython-input-291-bd945900e693>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  biddingData['discount%'] = biddingData.apply(lambda x: calDiscount(x['Intent_bids'][1],x['price_range']),axis=1)


In [292]:
biddingData['intents'] = biddingData['Intent_bids'].apply(lambda x: x[0])
biddingData.drop(columns=['price_range','Intent_bids'], inplace=True)

<ipython-input-292-7dc0f2adf26c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  biddingData['intents'] = biddingData['Intent_bids'].apply(lambda x: x[0])
/Users/kabirbatra/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [293]:
biddingData = biddingData[biddingData['discount%'] <= 50]
biddingData['discount%'].unique()

array([30., 25., 17., 10.,  5., 31.,  4., 19.,  6., 20.,  7.,  2., 13.,
       12., 32.,  8.,  0.,  3., 24., 16., 34., 11., 14., 15., 38., 18.,
       33., 22., 40., 35., 23., 27., 37., 28., 21., 26., 39.,  9.,  1.,
       44., 36., 43., 50., 47., 45., 41., 49., 29., 48., 42., 46.])

In [294]:
biddingData['discount%'] = biddingData['discount%'].apply(lambda x: int(x))

<ipython-input-294-9efe8038aa47>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  biddingData['discount%'] = biddingData['discount%'].apply(lambda x: int(x))


In [324]:
new_df = pd.DataFrame(columns=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
def insertRow(x):
    global new_df
    low = len(x)
    if len(x) < 18:
        for i in range(low,18):
            x.append(np.NaN)

    # dict = {0:x[0],1:x[1],2:x[2],3:x[3],4:x[4],5:x[5],6:x[6],6:x[6],7:x[7],8:x[8],9:x[9]}
    df = pd.DataFrame([x],columns=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
    print(df)
    new_df = pd.concat([new_df, df], ignore_index=True)

biddingData['intents'].apply(lambda x: insertRow(x))
#     arr = []))

      0              1              2      3   4   5   6   7   8   9   10  11  \
0  intro  counter-price  counter-price  agree NaN NaN NaN NaN NaN NaN NaN NaN   

   12  13  14  15  16  17  
0 NaN NaN NaN NaN NaN NaN  
      0         1              2      3      4   5   6   7   8   9   10  11  \
0  intro  disagree  counter-price  agree  agree NaN NaN NaN NaN NaN NaN NaN   

   12  13  14  15  16  17  
0 NaN NaN NaN NaN NaN NaN  
      0       1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16  \
0  intro  inform NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   

   17  
0 NaN  
      0              1      2   3   4   5   6   7   8   9   10  11  12  13  \
0  intro  counter-price  agree NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   

   14  15  16  17  
0 NaN NaN NaN NaN  
      0        1              2              3   4   5   6   7   8   9   10  \
0  intro  inquiry  counter-price  counter-price NaN NaN NaN NaN NaN NaN NaN   

   11  12  13  14  15  16  17  
0

0       None
1       None
3       None
6       None
16      None
        ... 
5240    None
5242    None
5243    None
5244    None
5246    None
Name: intents, Length: 2756, dtype: object

In [364]:
new_df.loc[:,13].unique()
new_df.drop(columns=[13,14,15,16,17],inplace=True)

In [367]:
from sklearn.tree import DecisionTreeClassifier

regressor = DecisionTreeClassifier(random_state = 0)
 
regressor.fit(new_df, biddingData['discount%'])

ValueError: could not convert string to float: 'intro'